In [16]:
import modeling.surname_classifier as cl
import modeling.surname_data as sd
import modeling.surname_common as sc
import torch.nn as nn
import torch



In [38]:
df_categories = sd.load_df_categories()
df_training, df_test = sd.load_df_surnames()

In [7]:
n_hidden = 128
n_categories = len(df_categories)
rnn = cl.RNN(sc.N_LETTERS, n_hidden, n_categories)

In [10]:
hidden =torch.zeros(1, n_hidden)

In [13]:
input = sc.letter_to_tensor('A')
output, next_hidden = rnn(input, hidden)

In [14]:
input = sc.surname_to_tensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)

In [17]:
criterion = nn.NLLLoss()

In [18]:
cl.LEARNING_RATE = 0.005 

In [19]:
def train(category_tensor, surname_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(surname_tensor.size()[0]):
        output, hidden = rnn(surname_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-cl.LEARNING_RATE, p.grad.data)

    return output, loss.item()

In [78]:
example = df_training.iloc[1]
surname_tensor = sc.surname_to_tensor(example['normalized'])
category_tensor = torch.tensor([int(df_categories[df_categories['category'] == example['category']]['index'])])
print(category_tensor)

tensor([16])


In [81]:
for iter in (0,10000):
    example = df_training.iloc[1]
    surname_tensor = sc.surname_to_tensor(example['normalized'])
    category_tensor = torch.tensor([int(df_categories[df_categories['category'] == example['category']]['index'])])
    train(category_tensor, surname_tensor)

In [83]:
cl.predict(rnn,'kim')

'[[-2.704958438873291, 9], [-2.750582456588745, 16], [-2.7949512004852295, 11]]'

In [84]:
torch.save(rnn,'data/model/rnn.pickle')